# C7.1

In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sympy as sp

## Distortion

In [2]:
df_par = pd.read_csv('Input/Par.csv')
par = dict(zip(df_par["Par"], df_par["Val"]))
par["A_hx_pergrid"] = par["Checker_x_p"]*par["SLM_d"]/par["Checker_A_x_g"] # x-orientation Side length per grid of Checker A
par["A_hy_pergrid"] = par["Checker_y_p"]*par["SLM_d"]/par["Checker_A_y_g"] # y-orientation Side length per grid of Checker A

par["B_hx_pergrid"] = par["Checker_x_p"]*par["SLM_d"]/par["Checker_B_x_g"] # x-orientation Side length per grid of Checker B
par["B_hy_pergrid"] = par["Checker_y_p"]*par["SLM_d"]/par["Checker_B_y_g"] # y-orientation Side length per grid of Checker B
par

{'Checker_x_p': 1024.0,
 'Checker_y_p': 768.0,
 'Checker_A_x_g': 8.0,
 'Checker_A_y_g': 6.0,
 'Checker_B_x_g': 64.0,
 'Checker_B_y_g': 48.0,
 'SLM_d': 26.0,
 'CCD_d': 3.2,
 'A_hx_pergrid': 3328.0,
 'A_hy_pergrid': 3328.0,
 'B_hx_pergrid': 416.0,
 'B_hy_pergrid': 416.0}

In [3]:
df2 = pd.read_csv('Input/Distortion.csv')

# ideal image height 
df2["X_h1"] = df2.apply(lambda x: x.X_g*par["A_hx_pergrid"]*x.beta if x.Checker == "A" else x.X_g*par["B_hx_pergrid"]*x.beta, axis=1) # X-orientation ideal image height 
df2["Y_h1"] = df2.apply(lambda x: x.Y_g*par["A_hy_pergrid"]*x.beta if x.Checker == "A" else x.Y_g*par["B_hy_pergrid"]*x.beta, axis=1) # Y-orientation ideal image height

df2["XY_h1x"] = df2.apply(lambda x: x.XY_g*par["A_hx_pergrid"]*x.beta if x.Checker == "A" else x.XY_g*par["B_hx_pergrid"]*x.beta, axis=1) # X-orientation ideal image height 
df2["XY_h1y"] = df2.apply(lambda x: x.XY_g*par["A_hy_pergrid"]*x.beta if x.Checker == "A" else x.XY_g*par["B_hy_pergrid"]*x.beta, axis=1) # Y-orientation ideal image height

# actual image height
df2["X_h2"] = (df2["X_x_p"]-df2["Center_x_p"])*par["CCD_d"] # X-orientation actual image height 
df2["Y_h2"] = -(df2["Y_y_p"]-df2["Center_y_p"])*par["CCD_d"] # Y-orientation actual image height

df2["XY_h2x"] = (df2["XY_x_p"]-df2["Center_x_p"])*par["CCD_d"] # XY-orientation actual image height (x comp.)
df2["XY_h2y"] = -(df2["XY_y_p"]-df2["Center_y_p"])*par["CCD_d"] # XY-orientation actual image height (y comp.)

# Diff
df2["X_dh"] = df2["X_h2"].abs() - df2["X_h1"].abs() # X-orientation Diff 
df2["Y_dh"] = df2["Y_h2"].abs() - df2["Y_h1"].abs() # Y-orientation Diff

df2["XY_dhx"] = df2["XY_h2x"].abs() - df2["XY_h1x"].abs() # XY-orientation Diff (x comp.)
df2["XY_dhy"] = df2["XY_h2y"].abs() - df2["XY_h1y"].abs() # XY-orientation Diff (y comp.)

# Distortion q
df2["X_q"] = (df2["X_dh"]/df2["X_h1"]).map(lambda x:format(x, '.2%')) # X-orientation Distortion q
df2["Y_q"] = (df2["Y_dh"]/df2["Y_h1"]).map(lambda x:format(x, '.2%')) # Y-orientation Distortion q

df2["XY_qx"] = (df2["XY_dhx"]/df2["XY_h1x"]).map(lambda x:format(x, '.2%')) # XY-orientation Distortion q (x comp.)
df2["XY_qy"] = (df2["XY_dhy"]/df2["XY_h1y"]).map(lambda x:format(x, '.2%')) # XY-orientation Distortion q (y comp.)

In [4]:
df2[["Lab", "X_dh", "Y_dh", "XY_dhx", "XY_dhy", "X_q", "Y_q", "XY_qx", "XY_qy"]]

,Lab,X_dh,Y_dh,XY_dhx,XY_dhy,X_q,Y_q,XY_qx,XY_qy
0,150_4,3.200,22.400,0.000,-25.600,0.19%,1.35%,0.00%,-2.05%
1,70_4,169.600,176.000,131.200,131.200,10.19%,10.58%,10.51%,10.51%
2,50_4,246.400,124.800,182.400,128.000,14.81%,7.50%,14.62%,10.26%
3,70_2,-57.600,-32.000,-38.400,19.200,-3.46%,-1.92%,-2.31%,1.15%
4,70_6,191.104,95.104,150.528,128.128,17.30%,8.61%,18.16%,15.46%
5,48x64_2X,140.800,137.600,83.200,96.000,8.46%,8.27%,6.67%,7.69%
6,48x64_4X,-422.400,-441.600,-422.400,-441.600,-25.38%,-26.54%,-25.38%,-26.54%


In [6]:
df2.to_csv('Output/Distortion.csv', index=False)